# Wrangling Former Colonies
## By: Scott Kustes

### Objective:
Compile a list of former colonies of modern countries. ADD INFO ABOUT WHAT DATA IS INCLUDED IN FINAL OUTPUT

#### Discussion:
History is complex. For many countries, the date of independence is not clear cut. This dataset contains Dr. Hensel's determination of independence date as the point at which a country makes its own foreign and domestic policy decisions. It also contains the independence dates as determined by The Correlates of War Project and the Gleditsch/Ward list of independent states for comparison. In some instances, there are vast differences between the dates.

#### Datasets:
The primary dataset was downloaded from <a href='http://www.paulhensel.org/icow.html' target='_new'>The Issue Correlates of War Project</a>. Additional datasets were downloaded from <a href='http://www.correlatesofwar.org' target='_new'>The Correlates of War Project</a>.  For specific dataset citations, see the References section at the bottom of the notebook.

#### Contents
<ul>
    <li><a href='#gather'>Data Gathering</a></li>
    <li><a href='#assess1'>Assess, Part 1</a></li>
    <li><a href='#clean1'>Clean, Part 1</a></li>
    <li><a href='#assess2'>Assess, Part 2</a></li>
    <li><a href='#clean2'>Clean, Part 2</a></li>
    <li><a href='#assess3'>Assess, Part 3</a></li>
    <li><a href='#clean3'>Clean, Part 3</a></li>
    <li><a href='#final'>Finished Dataframes</a></li>
    <li><a href='#references'>References</a></li>
</ul>

In [1]:
# Import packages
import requests
import pandas as pd
import os.path as os_path

<a id='gather'></a>
## Data Gathering
### Colonization Data

In [2]:
icow = pd.read_csv( 'colonial_data.csv' )
icow.sample(5)

,State,Name,ColRuler,IndFrom,IndDate,IndViol,IndType,SecFrom,SecDate,SecViol,Into,IntoDate,COWsys,GWsys,Notes
189,780,Sri Lanka (Ceylon),200,200,194802,0,2,-9,-9,-9,-9,-9,194802,194802,-9
102,404,Guinea-Bissau,235,235,197409,0,2,-9,-9,-9,-9,-9,197409,197409,-9
25,110,Guyana,200,200,196605,0,2,-9,-9,-9,-9,-9,196605,196605,Founded by Dutch but ceded to British in 1814
191,790,Nepal,-9,-9,176809,1,1,-9,-9,-9,-9,-9,192001,181601,-9
158,652,Syria,640,220,194604,1,2,651,196109,0,-9,-9,194604,194601,COW system membership interrupted by 1958-61 m...


### Country Codes
Read in and de-duplicate the country codes used by The Correlates of War Project. Rename columns and set `country_code` as the dataframe key.

In [3]:
country_codes = pd.read_csv( 'cow_country_codes.csv' )
country_codes.drop_duplicates( inplace=True )
country_codes.rename( columns={'StateAbb': 'abbreviation', 'CCode': 'country_code', 'StateNme': 'country_name'}, inplace=True )
country_codes.set_index( 'country_code', inplace=True )
country_codes.sample(5)

,abbreviation,country_name
country_code,,
666,ISR,Israel
260,GFR,German Federal Republic
771,BNG,Bangladesh
696,UAE,United Arab Emirates
55,GRN,Grenada


<a id='assess1'></a>
## Assess, Part 1

In [4]:
icow.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 217 entries, 0 to 216
Data columns (total 15 columns):
State       217 non-null int64
Name        217 non-null object
ColRuler    217 non-null int64
IndFrom     217 non-null int64
IndDate     217 non-null int64
IndViol     217 non-null int64
IndType     217 non-null int64
SecFrom     217 non-null int64
SecDate     217 non-null int64
SecViol     217 non-null int64
Into        217 non-null int64
IntoDate    217 non-null int64
COWsys      217 non-null int64
GWsys       217 non-null int64
Notes       217 non-null object
dtypes: int64(13), object(2)
memory usage: 25.5+ KB


In [5]:
# Unique values in Colonial Ruler column
print( icow['ColRuler'].nunique() )
icow['ColRuler'].unique()

17


array([200, 230, 220, 210, 235,  -9, 255, 300, 640, 365, 211, 325, 710,
       740, 900,   2, 920], dtype=int64)

In [6]:
icow['IndViol'].unique()

array([1, 0], dtype=int64)

In [7]:
icow['IndType'].unique()

array([2, 3, 1, 4], dtype=int64)

### Issues Found
`1)` Remove countries with -9 in ColRuler. These places were never colonized.

`2)` Drop columns related to secession ("Sec" columns) and merging into another country ("Into" columns).

`3)` Rename columns to more reader-friendly format.

<a id='clean1'></a>
## Clean, Part 1

### 1) Remove Uncolonized Countries
Delete all entries where ColRuler column is -9. This indicates a country that was never colonized.

#### Code

In [8]:
icow.drop( icow.query( 'ColRuler == -9' ).index, inplace=True )

#### Test

In [9]:
print( icow['ColRuler'].nunique() )
icow['ColRuler'].unique()

16


array([200, 230, 220, 210, 235, 255, 300, 640, 365, 211, 325, 710, 740,
       900,   2, 920], dtype=int64)

### Drop Unnecessary Columns
Drop columns related to secession from or merging into another country: `SecFrom`, `SecDate`, `SecViol`, `Into`, `IntoDate`

#### Code

In [10]:
icow.drop( columns=['SecFrom','SecDate','SecViol','Into','IntoDate'], axis=1, inplace=True )

#### Test

In [11]:
icow.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 171 entries, 0 to 216
Data columns (total 10 columns):
State       171 non-null int64
Name        171 non-null object
ColRuler    171 non-null int64
IndFrom     171 non-null int64
IndDate     171 non-null int64
IndViol     171 non-null int64
IndType     171 non-null int64
COWsys      171 non-null int64
GWsys       171 non-null int64
Notes       171 non-null object
dtypes: int64(8), object(2)
memory usage: 14.7+ KB


### 3) Rename Columns
Create reader-friendly column names.

#### Code

In [12]:
icow.rename( columns={
    'State': 'country_code',
    'Name': 'country',
    'ColRuler': 'colonizer',
    'IndFrom': 'indep_from',
    'IndDate': 'indep_date',
    'IndViol': 'indep_violent',
    'IndType': 'indep_type',
    'COWsys': 'cow_system_ind_date',
    'GWsys': 'gw_system_ind_date',
    'Notes': 'notes'
}, inplace=True )

#### Test

In [13]:
icow.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 171 entries, 0 to 216
Data columns (total 10 columns):
country_code           171 non-null int64
country                171 non-null object
colonizer              171 non-null int64
indep_from             171 non-null int64
indep_date             171 non-null int64
indep_violent          171 non-null int64
indep_type             171 non-null int64
cow_system_ind_date    171 non-null int64
gw_system_ind_date     171 non-null int64
notes                  171 non-null object
dtypes: int64(8), object(2)
memory usage: 14.7+ KB


<a id='assess2'></a>
## Assess, Part 2

In [14]:
print( icow['colonizer'].nunique() )
icow['colonizer'].unique()

16


array([200, 230, 220, 210, 235, 255, 300, 640, 365, 211, 325, 710, 740,
       900,   2, 920], dtype=int64)

In [15]:
print( icow['indep_from'].nunique() )
icow['indep_from'].unique()

32


array([200,   2, 220,  41, 230,  89, 100, 210, 235, 140, 255, 300, 315,
       640, 345, 365, 432, 211, 325, 530, 560,  -9, 625, 678, 710, 730,
       750, 770, 820, 850, 900, 920], dtype=int64)

### Issues Found
`1)` Add country name into dataframe for columns `colonizer` and `indep_from`.

`2)` Convert `indep_date`, `cow_system_ind_date`, and `gw_system_ind_date` from int to Time Period.

<a id='clean2'></a>
## Clean, Part 2

### 1) Add Country Name for columns with Country Codes
Insert columns to hold country name based on country codes in `colonizer` and `indep_from` columns. 

Five Central American countries were part of the Federal Republic of Central America (country code: 89), which is not in the Correlates of War Project list. Manually update these entries. Morocco and Saudi Arabia will remain as nan because indep_from is -9.

#### Code

In [16]:
columns = ['colonizer','indep_from']
for column in columns:
    # Join on country code, then rename the joined column
    icow = icow.join( country_codes['country_name'], on=column )
    new_column_name = column + '_name'
    icow.rename( columns={'country_name': new_column_name}, inplace=True )

In [17]:
indexes = icow.query( 'indep_from == 89' ).index.tolist()
icow.loc[ indexes, 'indep_from_name' ] = 'Federal Republic of Central America'

#### Test

In [18]:
icow.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 171 entries, 0 to 216
Data columns (total 12 columns):
country_code           171 non-null int64
country                171 non-null object
colonizer              171 non-null int64
indep_from             171 non-null int64
indep_date             171 non-null int64
indep_violent          171 non-null int64
indep_type             171 non-null int64
cow_system_ind_date    171 non-null int64
gw_system_ind_date     171 non-null int64
notes                  171 non-null object
colonizer_name         171 non-null object
indep_from_name        169 non-null object
dtypes: int64(8), object(4)
memory usage: 22.4+ KB


In [19]:
print( icow['colonizer'].nunique() )
print( icow['colonizer_name'].nunique() )
icow['colonizer_name'].unique()

16
16


array(['United Kingdom', 'Spain', 'France', 'Netherlands', 'Portugal',
       'Germany', 'Austria-Hungary', 'Turkey', 'Russia', 'Belgium',
       'Italy', 'China', 'Japan', 'Australia', 'United States of America',
       'New Zealand'], dtype=object)

In [20]:
print( icow['indep_from'].nunique() )
print( icow['indep_from_name'].nunique() )
icow['indep_from_name'].unique()

32
31


array(['United Kingdom', 'United States of America', 'France', 'Haiti',
       'Spain', 'Federal Republic of Central America', 'Colombia',
       'Netherlands', 'Portugal', 'Brazil', 'Germany', 'Austria-Hungary',
       'Czechoslovakia', 'Turkey', 'Yugoslavia', 'Russia', 'Mali',
       'Belgium', 'Italy', 'Ethiopia', 'South Africa', nan, 'Sudan',
       'Yemen Arab Republic', 'China', 'Korea', 'India', 'Pakistan',
       'Malaysia', 'Indonesia', 'Australia', 'New Zealand'], dtype=object)

In [21]:
icow[ icow['indep_from_name'].isnull() ]

,country_code,country,colonizer,indep_from,indep_date,indep_violent,indep_type,cow_system_ind_date,gw_system_ind_date,notes,colonizer_name,indep_from_name
148,600,Morocco,220,-9,166606,1,1,184701,181601,Independence interrupted by 1911-56 French occ...,France,NaN
162,670,Saudi Arabia,640,-9,192601,1,1,192705,193209,Created by Ibn Saud 1927; united Hejaz (freed ...,Turkey,NaN


### 2) Convert date columns to datetime

Convert `indep_date`, `cow_system_ind_date`, and `gw_system_ind_date` from int to Time Period. 

**Note to future me**: I originally attempted a conversion to datetime, but datetime has a limitation to dates between 1677-09-21 and 2262-04-11. A few values fall outside of this range, therefore the workaround found <a href='https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#timeseries-oob' target='_new'>here</a> was used. This caused the following problem: <a href='https://stackoverflow.com/questions/58019763/jupyter-kernel-crash-when-querying-dataframe-with-period-datatype' target='_new'>Jupyter kernel crash when querying dataframe with Period datatype</a>. 

*Solution:* The error was caused by -9 in the `gw_system_ind_date` column. While the conversion to Period didn't fail, it somehow corrupted the dataframe, making queries impossible. By setting -9 dates to 999901 prior to conversion, a valid Period of 9999-01-01 is obtained and the dataframe functions properly now.

#### Code

In [22]:
# Conversion function to create Time Period, copied without modification from Pandas documentation linked in the cell above
def conv(x):
    return pd.Period( year=x // 10000, month=x // 100 % 100, day=x % 100, freq='D' )

# -9 entries in date columns cause problems with the dataframe after conversion to Period
# This sets them to a valid far past date of 000101
def fix_dates(x):
    return '000101' if x == -9 else x

In [23]:
# Convert string representations of date to Time Period
# First, apply fix_dates function to each column to ensure missing data converts properly (-9 in the original dataset)
# Then append '01' to create format YYYYMMDD (dataset contains only YYYYMM), then apply conversion function
columns = ['indep_date', 'cow_system_ind_date', 'gw_system_ind_date']
for column in columns:
    icow[column] = icow[column].apply(fix_dates)
    icow[column] = ( icow[column].astype('str') + '01' ).astype('int').apply(conv)

#### Test

In [24]:
icow.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 171 entries, 0 to 216
Data columns (total 12 columns):
country_code           171 non-null int64
country                171 non-null object
colonizer              171 non-null int64
indep_from             171 non-null int64
indep_date             171 non-null period[D]
indep_violent          171 non-null int64
indep_type             171 non-null int64
cow_system_ind_date    171 non-null period[D]
gw_system_ind_date     171 non-null period[D]
notes                  171 non-null object
colonizer_name         171 non-null object
indep_from_name        169 non-null object
dtypes: int64(5), object(4), period[D](3)
memory usage: 22.4+ KB


In [25]:
# Check the three entries that failed when attempting conversion to datetime
icow[ icow.index.isin( [148,150,170] ) ][['country','indep_date','cow_system_ind_date','gw_system_ind_date']]

,country,indep_date,cow_system_ind_date,gw_system_ind_date
148,Morocco,1666-06-01,1847-01-01,1816-01-01
150,Tunisia,1591-01-01,1825-01-01,1816-01-01
170,Oman,1741-01-01,1971-10-01,1816-01-01


<a id='assess3'></a>
## Assess, Part 3

In [27]:
icow[ icow['colonizer'] != icow['indep_from'] ]

,country_code,country,colonizer,indep_from,indep_date,indep_violent,indep_type,cow_system_ind_date,gw_system_ind_date,notes,colonizer_name,indep_from_name
3,40,Cuba,230,2,1902-05-01,0,2,1902-05-01,1902-05-01,COW system interrupted 9/1906-1/1909 (US occup...,Spain,United States of America
5,42,Dominican Republic,230,41,1844-02-01,1,3,1894-01-01,1844-02-01,COW system interrupted 11/1916-9/1924 (US occu...,Spain,Haiti
17,90,Guatemala,230,89,1839-04-01,1,3,1868-01-01,1823-07-01,Seceded from United States/Provinces of Cent.A...,Spain,Federal Republic of Central America
18,91,Honduras,230,89,1838-11-01,1,3,1899-01-01,1840-01-01,Seceded from United States/Provinces of Cent.A...,Spain,Federal Republic of Central America
19,92,El Salvador,230,89,1840-04-01,1,3,1875-01-01,1840-01-01,Seceded from United States/Provinces of Cent.A...,Spain,Federal Republic of Central America
20,93,Nicaragua,230,89,1838-04-01,1,3,1900-01-01,1840-01-01,Seceded from United States/Provinces of Cent.A...,Spain,Federal Republic of Central America
21,94,Costa Rica,230,89,1838-11-01,1,3,1920-01-01,1840-01-01,Seceded from United States/Provinces of Cent.A...,Spain,Federal Republic of Central America
22,95,Panama,230,100,1903-11-01,1,3,1903-11-01,1903-11-01,Was part of Colombia until US supported 1903 r...,Spain,Colombia
24,101,Venezuela,230,100,1830-01-01,0,3,1841-01-01,1829-01-01,Seceded from Gran Colombia (not in COW system;...,Spain,Colombia
27,130,Ecuador,230,100,1830-05-01,0,3,1854-01-01,1830-05-01,Seceded from Gran Colombia (not in COW system;...,Spain,Colombia


<a id='clean3'></a>
## Clean, Part 3

### 1) 

#### Code

#### Test

<a id='references'></a>
## References

<li>Paul R. Hensel (2018). "ICOW Colonial History Data Set, version 1.1." Available at <a href='http://www.paulhensel.org/icowcol.html' target='_new'>http://www.paulhensel.org/icowcol.html</a></li>
<li><a href='http://www.correlatesofwar.org/data-sets/downloadable-files/cow-country-codes' target='_new'>Correlates of War country codes</a></li>